# Stem Separation with SoundLab

This notebook demonstrates how to use SoundLab's stem separation capabilities powered by Demucs.

**What you'll learn:**
- Load and configure different Demucs models
- Separate audio into individual stems (vocals, drums, bass, other)
- Work with separation results
- Export stems in different formats
- Best practices for stem separation

## Setup

First, let's import the necessary modules and configure logging.

In [ ]:
import soundlab
from soundlab.separation import StemSeparator, SeparationConfig, DemucsModel
from soundlab.io import load_audio, save_audio
from pathlib import Path

# For audio playback in notebooks
from IPython.display import Audio, display
import numpy as np

print(f"SoundLab version: {soundlab.__version__}")

## 1. Basic Stem Separation

Let's start with the simplest case - separating a song using the default model.

In [ ]:
# Initialize the separator with default settings
separator = StemSeparator()

# Path to your audio file
# Replace with your own audio file or use test fixtures
input_file = "../../tests/fixtures/audio/music_like_5s.wav"
output_dir = "./output/stems"

# Separate stems
print(f"Processing: {input_file}")
result = separator.separate(input_file, output_dir)

print(f"\nSeparation complete in {result.processing_time_seconds:.2f} seconds")
print(f"Available stems: {result.stem_names}")

### Accessing Individual Stems

The result object provides convenient properties to access each stem:

In [ ]:
# Access individual stems
print("Stem paths:")
print(f"  Vocals: {result.vocals}")
print(f"  Drums: {result.drums}")
print(f"  Bass: {result.bass}")
print(f"  Other: {result.other}")

# Or iterate over all stems
print("\nAll stems:")
for stem_name, stem_path in result.stems.items():
    print(f"  {stem_name}: {stem_path}")

### Play Back Individual Stems

Let's listen to each separated stem in the notebook:

In [ ]:
# Play vocals
if result.vocals and result.vocals.exists():
    print("🎤 Vocals:")
    display(Audio(str(result.vocals)))

# Play drums
if result.drums and result.drums.exists():
    print("\n🥁 Drums:")
    display(Audio(str(result.drums)))

# Play bass
if result.bass and result.bass.exists():
    print("\n🎸 Bass:")
    display(Audio(str(result.bass)))

# Play other (instrumental)
if result.other and result.other.exists():
    print("\n🎹 Other (Instrumental):")
    display(Audio(str(result.other)))

## 2. Choosing Different Models

SoundLab supports multiple Demucs models with different characteristics:

- **htdemucs**: Fast, good quality (default)
- **htdemucs_ft**: Fine-tuned, best quality (slower)
- **htdemucs_6s**: 6 stems including piano and guitar (experimental)
- **mdx_extra**: Alternative MDX architecture
- **mdx_extra_q**: Quantized MDX (faster, slightly lower quality)

Let's compare different models:

In [ ]:
# List all available models and their properties
print("Available Demucs Models:\n")
for model in DemucsModel:
    print(f"{model.value}:")
    print(f"  Description: {model.description}")
    print(f"  Stems: {model.stems}")
    print(f"  Stem count: {model.stem_count}")
    print()

### Using the Fine-Tuned Model (Best Quality)

In [ ]:
# Configure for best quality
config = SeparationConfig(
    model=DemucsModel.HTDEMUCS_FT,  # Fine-tuned model
    shifts=1,  # More shifts = better quality but slower
)

# Create separator with custom config
hq_separator = StemSeparator(config=config)

print(f"Using model: {config.model.value}")
print(f"Model description: {config.model.description}")

# Separate (uncomment to run)
# result_hq = hq_separator.separate(input_file, "./output/stems_hq")
# print(f"Processing time: {result_hq.processing_time_seconds:.2f}s")

### Using the 6-Stem Model (Piano and Guitar)

In [ ]:
# Configure for 6 stems
config_6s = SeparationConfig(
    model=DemucsModel.HTDEMUCS_6S,
)

separator_6s = StemSeparator(config=config_6s)

print(f"This model produces {config_6s.model.stem_count} stems:")
print(f"  {', '.join(config_6s.model.stems)}")

# Separate (uncomment to run)
# result_6s = separator_6s.separate(input_file, "./output/stems_6s")
# print(f"\nExtracted stems: {result_6s.stem_names}")

## 3. Advanced Configuration

Fine-tune the separation process with advanced parameters:

In [ ]:
# Advanced configuration
advanced_config = SeparationConfig(
    model=DemucsModel.HTDEMUCS_FT,
    
    # Quality settings
    segment_length=7.8,  # Length of segments (seconds, 1.0-30.0)
    overlap=0.25,        # Overlap between segments (0.1-0.9)
    shifts=1,            # Number of random shifts (0-5, higher = better quality)
    
    # Output format
    int24=True,          # Use 24-bit integer format
    float32=False,       # Use 32-bit float format
    mp3_bitrate=320,     # MP3 bitrate for .mp3 output (128-320)
    
    # Resource management
    device="auto",       # "auto", "cuda", or "cpu"
    split=True,          # Enable segment-based processing for long audio
)

print("Advanced Configuration:")
print(f"  Model: {advanced_config.model.value}")
print(f"  Segment length: {advanced_config.segment_length}s")
print(f"  Overlap: {advanced_config.overlap}")
print(f"  Shifts: {advanced_config.shifts}")
print(f"  Output: {'24-bit' if advanced_config.int24 else 'float32'}")
print(f"  Device: {advanced_config.device}")

## 4. Extracting Only Specific Stems

If you only need one stem (e.g., vocals), you can use the `two_stems` option for faster processing:

In [ ]:
# Extract only vocals (faster than extracting all stems)
vocals_only_config = SeparationConfig(
    model=DemucsModel.HTDEMUCS,
    two_stems="vocals",  # Options: "vocals", "drums", "bass", "other"
)

vocals_separator = StemSeparator(config=vocals_only_config)

# This will only output vocals and everything else (instrumental)
# result_vocals = vocals_separator.separate(input_file, "./output/vocals_only")
# print(f"Extracted stems: {result_vocals.stem_names}")

print("Tip: Use two_stems for faster processing when you only need one stem!")

## 5. Exporting Stems in Different Formats

After separation, you can convert stems to different audio formats:

In [ ]:
# Load a stem and save in different formats
if result.vocals and result.vocals.exists():
    # Load the vocals stem
    audio, sample_rate = load_audio(result.vocals)
    
    print(f"Loaded vocals: {audio.shape[0]} samples at {sample_rate} Hz")
    print(f"Duration: {audio.shape[0] / sample_rate:.2f} seconds")
    
    # Create output directory
    export_dir = Path("./output/exports")
    export_dir.mkdir(parents=True, exist_ok=True)
    
    # Export to different formats
    formats = {
        "wav": export_dir / "vocals.wav",
        "flac": export_dir / "vocals.flac",
        "mp3": export_dir / "vocals.mp3",
        "ogg": export_dir / "vocals.ogg",
    }
    
    for format_name, output_path in formats.items():
        save_audio(output_path, audio, sample_rate)
        file_size = output_path.stat().st_size / 1024  # KB
        print(f"  Exported to {format_name.upper()}: {file_size:.1f} KB")

## 6. Batch Processing Multiple Files

Process multiple audio files efficiently:

In [ ]:
from pathlib import Path

def batch_separate(input_dir, output_base_dir, config=None):
    """
    Separate stems for all audio files in a directory.
    
    Args:
        input_dir: Directory containing audio files
        output_base_dir: Base directory for outputs
        config: Optional SeparationConfig
    """
    separator = StemSeparator(config=config)
    input_path = Path(input_dir)
    
    # Find all audio files
    audio_extensions = ['.wav', '.mp3', '.flac', '.ogg', '.m4a']
    audio_files = []
    for ext in audio_extensions:
        audio_files.extend(input_path.glob(f'*{ext}'))
    
    print(f"Found {len(audio_files)} audio files\n")
    
    results = []
    for i, audio_file in enumerate(audio_files, 1):
        print(f"[{i}/{len(audio_files)}] Processing: {audio_file.name}")
        
        # Create output directory for this file
        output_dir = Path(output_base_dir) / audio_file.stem
        
        try:
            result = separator.separate(str(audio_file), str(output_dir))
            results.append(result)
            print(f"  ✓ Complete in {result.processing_time_seconds:.1f}s\n")
        except Exception as e:
            print(f"  ✗ Error: {e}\n")
            continue
    
    return results

# Example usage (uncomment to run):
# results = batch_separate(
#     input_dir="./audio_files",
#     output_base_dir="./output/batch",
#     config=SeparationConfig(model=DemucsModel.HTDEMUCS)
# )

print("Batch processing function defined!")

## Best Practices and Tips

### 1. Choosing the Right Model

- **htdemucs**: Good balance of speed and quality for most use cases
- **htdemucs_ft**: Best quality, use for final production
- **htdemucs_6s**: When you need piano/guitar separation (experimental)
- **mdx_extra_q**: When speed is critical and quality can be compromised

### 2. Performance Optimization

- Use `two_stems` when you only need one stem (2x faster)
- Enable GPU acceleration with `device="cuda"` if available
- For long files, keep `split=True` to avoid memory issues
- Reduce `shifts` parameter for faster processing

### 3. Quality Optimization

- Increase `shifts` to 1-2 for better quality (slower)
- Use `htdemucs_ft` model for best results
- Keep `segment_length` at default (7.8s) for optimal quality
- Increase `overlap` to 0.3-0.5 for smoother results

### 4. Output Format

- Use `int24=True` for lossless quality with reasonable file size
- Use `float32=True` for maximum precision (larger files)
- For MP3 output, use `mp3_bitrate=320` for best quality

### 5. Common Issues

- **Out of memory**: Reduce `segment_length` or use `split=True`
- **Slow processing**: Use faster model or reduce `shifts`
- **Poor quality**: Try `htdemucs_ft` with `shifts=1`
- **GPU not detected**: Check CUDA installation and use `device="cuda"`

## Summary

In this notebook, you learned how to:

✓ Perform basic stem separation with default settings  
✓ Choose and configure different Demucs models  
✓ Access and play back individual stems  
✓ Use advanced configuration options  
✓ Extract specific stems for faster processing  
✓ Export stems in different audio formats  
✓ Batch process multiple audio files  
✓ Apply best practices for quality and performance  

**Next Steps:**
- Explore MIDI transcription in notebook 02
- Learn audio analysis techniques in notebook 03
- Apply effects processing in notebook 04